<a href="https://colab.research.google.com/github/pvfalcao/A-semi-supervised-learning-approach-with-data-drift-detection-to-deal-with-ransomware/blob/main/30s/Rel_RASCO_with_data_drift_30S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas


In [ ]:
!pip install sslearn
!pip install frouros

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [ ]:
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sslearn
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sslearn.wrapper import RelRasco
from sslearn.model_selection import artificial_ssl_dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from frouros.callbacks.batch import PermutationTestDistanceBased
from frouros.detectors.data_drift import MMD
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
from frouros.detectors.data_drift import KSTest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


Recebendo conjunto de dados


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Navarra/N30S30/N30S30.csv', header = None)
zero = pd.read_csv(r'/content/drive/MyDrive/Navarra/N30S30/zeroDays.csv', header = None)
user = pd.read_csv(r'/content/drive/MyDrive/Navarra/N30S30/userSamples_train.csv', header = None)

targets = df.iloc[:,df.shape[1] - 1]
df = df.iloc[:,0:-1]
tz = zero.iloc[:,zero.shape[1] - 1]
zero = zero.iloc[:,0:-1]
tu = user.iloc[:,user.shape[1] - 1]
user = user.iloc[:,0:-1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Modelo

In [ ]:

#Initialize the variables

precision_list = []
recall_list = []
f1_list = []
accuracy_list = []

# Initialize 10-Fold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = RelRasco()

#Consumption metrics

for lr in [0.1, 0.2, 0.3, 0.4]:

  psutil.cpu_percent(interval=None)
  start = time.process_time()

  # Split the dataset into 10 folds

  for fold, (train_idx, test_idx) in enumerate(skf.split(df, targets)):

      # Select data using .iloc to get pandas DataFrames/Series
      train_features = df.iloc[train_idx]
      train_labels = targets.iloc[train_idx]
      test_features = df.iloc[test_idx]
      test_labels = targets.iloc[test_idx]
      TP = 0
      FP = 0
      TN = 0
      FN = 0
      counter = 0
      precision_list = []
      recall_list = []
      f1_list = []
      accuracy_list = []


      X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
      y_train, y_test = targets.iloc[train_idx], targets.iloc[test_idx]

      for i in range(X_train.shape[1]):
        feature_idx = i
        alpha = 0.01
        # Define and fit detector
        detector = KSTest()

        X_train_numeric = pd.to_numeric(X_train.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        __ = detector.fit(X_train_numeric)
            # Convert the feature column to numeric before comparing
        X_test_numeric = pd.to_numeric(X_test.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        result, _ = detector.compare(X_test_numeric)

        # Check if drift is taking place
        if result.p_value <= alpha:
            counter += 1
        if counter>(X_train.shape[1]/2):
          print("More than half of features had data drift detected.")

          X_add = X_test.sample(frac=0.2, random_state=42)
          y_add = y_test.sample(frac=0.2, random_state=42)
          X_train = pd.concat([X_train, X_add])
          y_train = pd.concat([y_train, y_add])
          X_test = X_test.drop(X_add.index)
          y_test = y_test.drop(y_add.index)
          print("Datasets updated")
          break



      X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
      clf.fit(X_ss, y_ss)

        # Unlabeled score
      print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")




      y_pred = clf.predict(X_test)

      TN, FP, FN, TP = confusion_matrix(y_pred, y_test).ravel()

      precision = TP / (TP + FP) if TP + FP > 0 else 0
      recall = TP / (TP + FN) if TP + FN > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
      accuracy = (TN + TP)/(TP + FP + FN + TN)


      accuracy_list.append(accuracy)
      precision_list.append(precision)
      recall_list.append(recall)
      f1_list.append(f1)



  #Consumption measures
  end = time.process_time()
  print('The processing time was:', end - start)
  cpu_usage_system = psutil.cpu_percent(interval=None)
  print(f"System-wide CPU usage: {cpu_usage_system}%")
  print(f"Load average:{psutil.getloadavg()}")
  print(f"CPU Frequency:{psutil.cpu_freq()}")
  print("CPU usage (%):", psutil.cpu_percent())
  print("CPU usage (%):", psutil.cpu_percent())
  ram = psutil.virtual_memory()
  print("RAM usage (%):", ram.percent)
  print("RAM used (GB):", round(ram.used / 1e9, 2))
  mean_precision = sum(precision_list) / len(precision_list)
  mean_recall = sum(recall_list) / len(recall_list)
  mean_f1 = sum(f1_list) / len(f1_list)
  mean_accuracy = sum(accuracy_list) / len(accuracy_list)
  print(f'Mean precision: {mean_precision:.5f}')
  print(f'Mean recall: {mean_recall:.5f}')
  print(f'Mean F1 Score: {mean_f1:.5f}')
  print(f'Mean accuracy: {mean_accuracy:.5f}')







[LR=0.1] Unlabeled score: 0.9957
[LR=0.1] Unlabeled score: 0.9949
[LR=0.1] Unlabeled score: 0.9958
[LR=0.1] Unlabeled score: 0.9959
[LR=0.1] Unlabeled score: 0.9970
[LR=0.1] Unlabeled score: 0.9960
[LR=0.1] Unlabeled score: 0.9961
[LR=0.1] Unlabeled score: 0.9973
[LR=0.1] Unlabeled score: 0.9959
[LR=0.1] Unlabeled score: 0.9962
The processing time was: 94.41407713699999
System-wide CPU usage: 67.0%
Load average:(1.41259765625, 1.017578125, 0.47998046875)
CPU Frequency:scpufreq(current=2199.998, min=0.0, max=0.0)
CPU usage (%): 0.0
CPU usage (%): 0.0
RAM usage (%): 14.1
RAM used (GB): 1.59
Mean precision: 0.99379
Mean recall: 0.99379
Mean F1 Score: 0.99379
Mean accuracy: 0.99669
[LR=0.2] Unlabeled score: 0.9969
[LR=0.2] Unlabeled score: 0.9967
[LR=0.2] Unlabeled score: 0.9961
[LR=0.2] Unlabeled score: 0.9971
[LR=0.2] Unlabeled score: 0.9978
[LR=0.2] Unlabeled score: 0.9977
[LR=0.2] Unlabeled score: 0.9962
[LR=0.2] Unlabeled score: 0.9977
[LR=0.2] Unlabeled score: 0.9972
[LR=0.2] Unlabel

In [ ]:
for i in range(X_train.shape[1]):
 feature_idx = i
 alpha = 0.01
 # Define and fit detector
 detector = KSTest()

 X_train_numeric = pd.to_numeric(X_train.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
 __ = detector.fit(X_train_numeric)
    # Convert the feature column to numeric before comparing
 X_test_numeric = pd.to_numeric(X_test.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
 result, _ = detector.compare(X_test_numeric)

 # Check if drift is taking place
 if result.p_value <= alpha:
     counter += 1


Zero day dataset

In [ ]:
for lr in [0.1, 0.2, 0.3, 0.4]:
  print(lr)

0.1
0.2
0.3
0.4


In [ ]:

counter = 0
for i in range(X_train.shape[1]):
        feature_idx = i
        alpha = 0.01
        # Define and fit detector
        detector = KSTest()

        X_train_numeric = pd.to_numeric(X_train.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        __ = detector.fit(X_train_numeric)
            # Convert the feature column to numeric before comparing
        X_test_numeric = pd.to_numeric(zero.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        result, _ = detector.compare(X_test_numeric)

        # Check if drift is taking place
        if result.p_value <= alpha:
            counter += 1
        if counter>(X_train.shape[1]/2):
          print("More than half of features had data drift detected.")

          X_add = zero.sample(frac=0.2, random_state=42)
          y_add = tz.sample(frac=0.2, random_state=42)
          X_train = pd.concat([X_train, X_add])
          y_train = pd.concat([y_train, y_add])
          X_test = zero.drop(X_add.index)
          y_test = tz.drop(y_add.index)
          print("Datasets updated")
          break
for lr in [0.1, 0.2, 0.3, 0.4]:
   # Reset TP, FP, FN, TN for each LR value
   TP = 0
   FP = 0
   FN = 0
   TN = 0

   X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
   clf.fit(X_ss, y_ss)

   # Unlabeled score
   print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")
   y_pred = clf.predict(X_test)
   for i in range(y_test.shape[0]):
       if y_pred[i] == 0 and tz[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tz[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tz[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tz[i] == 1:
          TP += 1
   precision = TP / (TP + FP) if TP + FP > 0 else 0
   recall = TP / (TP + FN) if TP + FN > 0 else 0
   f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
   accuracy = (TN + TP)/(TP + FP + FN + TN)
   print(f'The results for {lr} are:')
   print(f'Precision: {precision:.5f}')
   print(f'Recall: {recall:.5f}')
   print(f'F1: {f1:.5f}')
   print(f'Accuracy: {accuracy:.5f}')



More than half of features had data drift detected.
Datasets updated
[LR=0.1] Unlabeled score: 0.9940
The results for 0.1 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.2] Unlabeled score: 0.9969
The results for 0.2 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.3] Unlabeled score: 0.9984
The results for 0.3 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000
[LR=0.4] Unlabeled score: 0.9979
The results for 0.4 are:
Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000


User sample dataset

In [ ]:

counter = 0
for i in range(X_train.shape[1]):
        feature_idx = i
        alpha = 0.01
        # Define and fit detector
        detector = KSTest()

        X_train_numeric = pd.to_numeric(X_train.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        __ = detector.fit(X_train_numeric)
            # Convert the feature column to numeric before comparing
        X_test_numeric = pd.to_numeric(user.iloc[:, feature_idx], errors='coerce').to_numpy().reshape(-1, 1)
        result, _ = detector.compare(X_test_numeric)

        # Check if drift is taking place
        if result.p_value <= alpha:
            counter += 1
        if counter>(X_train.shape[1]/2):
          print("More than half of features had data drift detected.")

          X_add = user.sample(frac=0.2, random_state=42)
          y_add = tu.sample(frac=0.2, random_state=42)
          X_train = pd.concat([X_train, X_add])
          y_train = pd.concat([y_train, y_add])
          X_test = user.drop(X_add.index)
          y_test = tu.drop(y_add.index)
          print("Datasets updated")
          break
for lr in [0.1, 0.2, 0.3, 0.4]:
   # Reset TP, FP, FN, TN for each LR value
   TP = 0
   FP = 0
   FN = 0
   TN = 0

   X_ss, y_ss, X_unl, y_unl = artificial_ssl_dataset(X_train, y_train, lr)
   clf.fit(X_ss, y_ss)

   # Unlabeled score
   print(f"[LR={lr}] Unlabeled score: {clf.score(X_unl, y_unl):.4f}")
   y_pred = clf.predict(X_test)
   for i in range(y_test.shape[0]):
       if y_pred[i] == 0 and tu[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tu[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tu[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tu[i] == 1:
          TP += 1
   precision = TN / (TN + FN) if (TN + FN) > 0 else 0
   recall = TN / (TN + FP) if (TN + FP) > 0 else 0
   f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
   accuracy = (TN + TP)/(TP + FP + FN + TN)
   print(f'The results for {lr} are:')
   print(f'Precision: {precision:.5f}')
   print(f'Recall: {recall:.5f}')
   print(f'F1: {f1:.5f}')
   print(f'Accuracy: {accuracy:.5f}')



More than half of features had data drift detected.
Datasets updated
[LR=0.1] Unlabeled score: 0.9891
The results for 0.1 are:
Precision: 1.00000
Recall: 0.99925
F1: 0.99962
Accuracy: 0.99925
[LR=0.2] Unlabeled score: 0.9951
The results for 0.2 are:
Precision: 1.00000
Recall: 0.99934
F1: 0.99967
Accuracy: 0.99934
[LR=0.3] Unlabeled score: 0.9955
The results for 0.3 are:
Precision: 1.00000
Recall: 0.99953
F1: 0.99977
Accuracy: 0.99953
[LR=0.4] Unlabeled score: 0.9978
The results for 0.4 are:
Precision: 1.00000
Recall: 0.99991
F1: 0.99995
Accuracy: 0.99991


In [ ]:
TN

19773

In [ ]:
FP

13

In [ ]:
TP

0

In [ ]:
FN

0

In [ ]:

TP = 0
FP = 0
FN = 0
TN = 0


y_pred = clf.predict(user)

TN, FP, FN, TP = confusion_matrix(y_pred, tu).ravel()

precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TN + TP)/(TP + FP + FN + TN)

print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')
